In [1]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkFiles
from pyspark.ml.feature import CountVectorizer

import numpy as np

schema = StructType([
    StructField("market_name", StringType(), True),
    StructField("vendor_name", StringType(), True),
    StructField("price", StringType(), True),
    StructField("item_name", StringType(), True),
    StructField("ships_from", StringType(), True),
    StructField("description", StringType(), True),
    StructField("date", StringType(), True),
    StructField("adjusted_price", StringType(), True)
])

files = 's3://15405finalprojectcsvdata/combined_and_cleaned_data/part-00000-c6f08ba4-b88b-4295-8d93-7a9f118df9db-c000.csv'

df = spark.read.csv(files, multiLine=True,header=False, mode="DROPMALFORMED", schema=schema)



df = df.drop('market_name', 'vendor_name', 'price', 'item_name', 'ships_from', 'date', 'adjusted_price')

df.show()

'''
mask = np.random.rand(len(df)) < 0.8

temp_train_df = df[mask]
temp_test_df = df[~mask]

train_docs = list(temp_train_df.body)
train_tags = list(temp_train_df.label - 1)

test_docs = list(temp_test_df.body)
test_tags = list(temp_test_df.label - 1)
'''



Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1586656534431_0005,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.
+--------------------+
|         description|
+--------------------+
|MORE DETAILED PIC...|
|Generic viagra si...|
|VERY POTENT Tobac...|
|-----------------...|
|6 tabs of temazep...|
|This order is for...|
|This listing is f...|
|This listing is f...|
|We offer 2g of ou...|
|We offer 5g of ou...|
|We offer 50g of o...|
|3x 100mg seroquel...|
|7g Psilocybe cube...|
|This listing is f...|
|****** HOLIDAY SA...|
|****** HOLIDAY SA...|
|Original looking ...|
|Great quality (as...|
|We offer great qu...|
|Consists of clean...|
+--------------------+
only showing top 20 rows

'\nmask = np.random.rand(len(df)) < 0.8\n\ntemp_train_df = df[mask]\ntemp_test_df = df[~mask]\n\ntrain_docs = list(temp_train_df.body)\ntrain_tags = list(temp_train_df.label - 1)\n\ntest_docs = list(temp_test_df.body)\ntest_tags = list(temp_test_df.label - 1)\n'

In [2]:
train_df, test_df = df.randomSplit([0.8, 0.2])

train_df = train_df.where(col('description').isNotNull())
test_df = test_df.where(col('description').isNotNull())

#small = df.sample(False, .001).where(col('description').isNotNull())

#small.show()

In [3]:
cv = CountVectorizer(inputCol="description", outputCol="bow", vocabSize=10000, minDF=3.0, maxDF=0.8)

In [4]:
def proc_dataframe(df):
    spl = df.select(split(col('description'), ' ').alias('description'))
    model = cv.fit(spl)
    result = model.transform(spl).select('bow')
    return result.withColumn("doc_id", monotonically_increasing_id()).withColumn("label", lit(0))
    

In [5]:
#small_result = proc_dataframe(small)
train_result = proc_dataframe(train_df)
test_result = proc_dataframe(test_df)



In [8]:

train_final = train_result.rdd.repartition(1).toDF()
test_final = test_result.rdd.repartition(1).toDF()


In [10]:
# write to s3
#train_final.write.parquet('s3://15405finalprojectcsvdata/split_dataset/train_df.parquet')
test_final.write.parquet('s3://15405finalprojectcsvdata/split_dataset/test_df.parquet')

